<a href="https://colab.research.google.com/github/LittleQili/CS473/blob/main/my_mesh_segmentation_demo_indrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 Google LLC.


In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/mesh_seg

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/mesh_seg


# Mesh Segmentation using Feature Steered Graph Convolutions
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/graphics/blob/master/tensorflow_graphics/notebooks/mesh_segmentation_demo.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/graphics/blob/master/tensorflow_graphics/notebooks/mesh_segmentation_demo.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [ ]:
!pip install tensorflow_graphics
# !pip install tensorboardcolab

In [ ]:
# from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
# tbc=TensorBoardColab()

In [3]:
import glob
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from tensorflow_graphics.nn.layer import graph_convolution as graph_conv
from tensorflow_graphics.notebooks import mesh_segmentation_dataio as dataio
from tensorflow_graphics.notebooks import mesh_viewer

Instructions for updating:
non-resource variables are not supported in the long term


Note this notebook works best in Graph mode.

### Fetch model files and data

For convenience, we provide a pre-trained model. Let's now download a pre-trained model checkpoint and the test data. The meshes are generated using Unity Multipurpose Avatar system [UMA](https://assetstore.unity.com/packages/3d/characters/uma-2-unity-multipurpose-avatar-35611).

In [4]:
# mydrivedatafolder = '/content/drive/MyDrive/mesh_seg/data/'
mydrivedatafolder = ''
path_to_model_zip = tf.keras.utils.get_file(
    mydrivedatafolder+'model.zip',
    origin='https://storage.googleapis.com/tensorflow-graphics/notebooks/mesh_segmentation/model.zip',
    extract=True)

path_to_data_zip = tf.keras.utils.get_file(
    mydrivedatafolder+'data.zip',
    origin='https://storage.googleapis.com/tensorflow-graphics/notebooks/mesh_segmentation/data.zip',
    extract=True)

local_model_dir = os.path.join(os.path.dirname(path_to_model_zip), 'model')
# print result: ['/root/.keras/datasets/data/Dancer_test_sequence.tfrecords'] 只是一个文件路径而已
test_data_files = [
    os.path.join(
        os.path.dirname(path_to_data_zip),
        'data/Dancer_test_sequence.tfrecords')
]
print(test_data_files)

['/root/.keras/datasets/data/Dancer_test_sequence.tfrecords']


## Load and visualize test data

For graph convolutions, we need a *weighted adjacency matrix* denoting the mesh
connectivity. Feature-steered graph convolutions expect self-edges in the mesh
connectivity for each vertex, i.e. the diagonal of the weighted adjacency matrix
should be non-zero. This matrix is defined as:
```
A[i, j] = w[i,j] if vertex i and vertex j share an edge,
A[i, i] = w[i,i] for each vertex i,
A[i, j] = 0 otherwise.
where, w[i, j] = 1/(degree(vertex i)), and sum(j)(w[i,j]) = 1
```
Here degree(vertex i) is the number of edges incident on a vertex (including the
self-edge). This weighted adjacency matrix is stored as a SparseTensor.

We will load the test meshes from the test [tf.data.TFRecordDataset](https://www.tensorflow.org/api_docs/python/tf/data/TFRecordDataset)
downloaded above. Each mesh is stored as a
[tf.Example](https://www.tensorflow.org/api_docs/python/tf/train/Example), with
the following fields:

*   'num_vertices', V: Number of vertices in each mesh.
*   'num_triangles', T: Number of triangles in each mesh.
*   'vertices': A [V, 3] float tensor of vertex positions.
*   'triangles': A [T, 3] integer tensor of vertex indices for each triangle.
*   'labels': A [V] integer tensor with segmentation class label for each
    vertex.

As
each mesh may have a varying number of vertices and faces (and the corresponding
connectivity matrix), we pad the data tensors with '0's in each batch.

For details on the dataset pipeline implementation, take a look at
mesh_segmentation_dataio.py.

Let's try to load a batch from the test TFRecordDataset, and visualize the first
mesh with each vertex colored by the part label.

## Model Definition

Given a mesh with V vertices and D-dimensional per-vertex input features (e.g.
vertex position, normal), we would like to create a network capable of
classifying each vertex to a part label. Let's first create a mesh encoder that
encodes each vertex in the mesh into C-dimensional logits, where C is the number
of parts. First we use 1x1 convolutions to change input feature dimensions,
followed by a sequence of feature steered graph convolutions and ReLU
non-linearities, and finally 1x1 convolutions to logits, which are used for
computing softmax cross entropy as described below.

Note that this model does not use any form of pooling, which is outside the scope of this notebook.

![](https://storage.googleapis.com/tensorflow-graphics/notebooks/mesh_segmentation/mesh_segmentation_model_def.png)

In [5]:
MODEL_PARAMS = {
    'num_filters': 8,
    'num_classes': 16,
    'encoder_filter_dims': [32, 64, 128],
}


def mesh_encoder(batch_mesh_data, num_filters, output_dim, conv_layer_dims):
  """A mesh encoder using feature steered graph convolutions.

    The shorthands used below are
      `B`: Batch size.
      `V`: The maximum number of vertices over all meshes in the batch.
      `D`: The number of dimensions of input vertex features, D=3 if vertex
        positions are used as features.

  Args:
    batch_mesh_data: A mesh_data dict with following keys
      'vertices': A [B, V, D] `float32` tensor of vertex features, possibly
        0-padded.
      'neighbors': A [B, V, V] `float32` sparse tensor of edge weights.
      'num_vertices': A [B] `int32` tensor of number of vertices per mesh.
    num_filters: The number of weight matrices to be used in feature steered
      graph conv.
    output_dim: A dimension of output per vertex features.
    conv_layer_dims: A list of dimensions used in graph convolution layers.

  Returns:
    vertex_features: A [B, V, output_dim] `float32` tensor of per vertex
      features.
  """
  batch_vertices = batch_mesh_data['vertices']

  # Linear: N x D --> N x 16.
  vertex_features = tf.keras.layers.Conv1D(16, 1, name='lin16')(batch_vertices)
  res_vertex_features = vertex_features

  # graph convolution layers
  for dim in conv_layer_dims:
    with tf.variable_scope('conv_%d' % dim):
      vertex_features = graph_conv.feature_steered_convolution_layer(
          vertex_features,
          batch_mesh_data['neighbors'],
          batch_mesh_data['num_vertices'],
          num_weight_matrices=num_filters,
          num_output_channels=dim)
    vertex_features = tf.nn.relu(vertex_features)

  # Linear: N x 128 --> N x 256.
  vertex_features = tf.keras.layers.Conv1D(
      256, 1, name='lin256')(
          vertex_features)
  vertex_features = tf.nn.relu(vertex_features)

  # Linear: N x 256 --> N x output_dim.
  vertex_features = tf.keras.layers.Conv1D(
      output_dim, 1, name='lin_output')(
          vertex_features)
  vertex_features = vertex_features + res_vertex_features

  return vertex_features

Given a mesh encoder, let's define a model_fn for a custom
[tf.Estimator](https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator)
for vertex classification using softmax cross entropy loss. A tf.Estimator model_fn returns the ops necessary to perform training, evaluation, or predictions given inputs and a number of other parameters. Recall that the
vertex tensor may be zero-padded (see Dataset Pipeline above), hence we must mask out the contribution from the padded values.

In [6]:
def get_learning_rate(params):
  """Returns a decaying learning rate."""
  global_step = tf.train.get_or_create_global_step()
  learning_rate = tf.train.exponential_decay(
      params['init_learning_rate'],
      global_step,
      params['lr_decay_steps'],
      params['lr_decay_rate'])
  return learning_rate

def model_fn(features, labels, mode, params):
  """Returns a mesh segmentation model_fn for use with tf.Estimator."""
  logits = mesh_encoder(features, params['num_filters'], params['num_classes'],
                        params['encoder_filter_dims'])
  predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
  outputs = {
      'vertices': features['vertices'],
      'triangles': features['triangles'],
      'num_vertices': features['num_vertices'],
      'num_triangles': features['num_triangles'],
      'predictions': predictions,
  }
  # For predictions, return the outputs.
  if mode == tf.estimator.ModeKeys.PREDICT:
    outputs['labels'] = features['labels']
    return tf.estimator.EstimatorSpec(mode=mode, predictions=outputs)
  # Loss
  # Weight the losses by masking out padded vertices/labels.
  vertex_ragged_sizes = features['num_vertices']
  mask = tf.sequence_mask(vertex_ragged_sizes, tf.shape(labels)[-1])
  loss_weights = tf.cast(mask, dtype=tf.float32)
  loss = tf.losses.sparse_softmax_cross_entropy(
      logits=logits, labels=labels, weights=loss_weights)
  # For training, build the optimizer.
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(
        learning_rate=get_learning_rate(params),
        beta1=params['beta'],
        epsilon=params['adam_epsilon'])
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
      train_op = optimizer.minimize(
          loss=loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # For eval, return eval metrics.
  eval_ops = {
      'mean_loss':
          tf.metrics.mean(loss),
      'accuracy':
          tf.metrics.accuracy(
              labels=labels, predictions=predictions, weights=loss_weights)
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_ops)

## Train the model from scratch

Now let's train the mesh segmentation model from scratch. First we will download the train dataset files, and use tf.Estimator.train_and_evaluate to train a model.

Note: Training code is provided inside colab for demonstration, and may be slow. For optimal performance, consider running the training process as a command line process, and a tensorboard process to track.

In [7]:
path_to_train_data_zip = tf.keras.utils.get_file(
    mydrivedatafolder+'train_data.zip',
    origin='https://storage.googleapis.com/tensorflow-graphics/notebooks/mesh_segmentation/train_data.zip',
    extract=True)

train_data_files = glob.glob(
    os.path.join(os.path.dirname(path_to_train_data_zip), '*train*.tfrecords'))

retrain_model_dir = os.path.join(local_model_dir, 'retrain')

In [10]:
train_io_params = {
    'batch_size': 8,
    'parallel_threads': 8,
    'is_training': True,
    'shuffle': True,
    'sloppy': True,
}

eval_io_params = {
    'batch_size': 8,
    'parallel_threads': 8,
    'is_training': False,
    'shuffle': False
}


def train_fn():
  return dataio.create_input_from_dataset(dataio.create_dataset_from_tfrecords,
                                          train_data_files, train_io_params)


def eval_fn():
  return dataio.create_input_from_dataset(dataio.create_dataset_from_tfrecords,
                                          test_data_files, eval_io_params)


train_params = {
    'beta': 0.9,
    'adam_epsilon': 1e-8,
    'init_learning_rate': 0.001,
    'lr_decay_steps': 1000,
    'lr_decay_rate': 0.95,
}

train_params.update(MODEL_PARAMS)

# about 15 steps one save and evaluation.
# checkpoint_delay = 20  # Checkpoint every 2 minutes.
eval_spec_delay_secs = 20
eval_steps = 15
save_checkpoints_steps = 100
# save_summary_steps = 20
max_steps = 2000  # Number of training steps.
log_step_count_steps = 2

config = tf.estimator.RunConfig(
    log_step_count_steps=log_step_count_steps,
    save_checkpoints_secs=eval_spec_delay_secs,
    # save_checkpoints_steps=save_checkpoints_steps,
    # save_summary_steps = save_summary_steps, # I cannot find it where.
    keep_checkpoint_max=5)

classifier = tf.estimator.Estimator(
    model_fn=model_fn,
    model_dir=retrain_model_dir,
    config=config,
    params=train_params)
train_spec = tf.estimator.TrainSpec(input_fn=train_fn, max_steps=max_steps)
eval_spec = tf.estimator.EvalSpec(
    input_fn=eval_fn,
    steps=eval_steps,
    start_delay_secs=eval_spec_delay_secs,# evaluation time delays
    throttle_secs=eval_spec_delay_secs/2 # do not evaluate after within this parameter seconds after the latest evaluation.
    )

print('Start training & eval.')
tf.estimator.train_and_evaluate(classifier, train_spec, eval_spec)
print('Train and eval done.')

INFO:tensorflow:Using config: {'_model_dir': '/root/.keras/datasets/model/retrain', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 20, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 2, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Start training & eval.
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running traini

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/conv_128/graph_convolution_feature_steered_convolution_weights/graph_convolution_feature_steered_convolution/GatherV2_grad/Reshape_1:0", shape=(?,), dtype=int64), values=Tensor("gradients/conv_128/graph_convolution_feature_steered_convolution_weights/graph_convolution_feature_steered_convolution/GatherV2_grad/Reshape:0", shape=(?, 8), dtype=float32), dense_shape=Tensor("gradients/conv_128/graph_convolution_feature_steered_convolution_weights/graph_convolution_feature_steered_convolution/GatherV2_grad/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(Indexe

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /root/.keras/datasets/model/retrain/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 2.7600887, step = 1
INFO:tensorflow:global_step/sec: 0.375511
INFO:tensorflow:loss = 2.7234569, step = 3 (5.328 sec)
INFO:tensorflow:global_step/sec: 0.543413
INFO:tensorflow:loss = 2.695843, step = 5 (3.679 sec)
INFO:tensorflow:global_step/sec: 0.551321
INFO:tensorflow:loss = 2.6563354, step = 7 (3.628 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 8...
INFO:tensorflow:Saving checkpoints for 8 into /root/.keras/datasets/model/retrain/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after s

KeyboardInterrupt: ignored

In [ ]:
# classifier.eval_dir()

'/root/.keras/datasets/model/retrain/eval'

In [9]:
%rm -rf /root/.keras/datasets/model
%cd /root/.keras/datasets
%ls -a

/root/.keras/datasets
./                        Dancer_train_30.tfrecords  data.zip
../                       Dancer_train_31.tfrecords  model.zip
Dancer_train_1.tfrecords  Dancer_train_32.tfrecords  train_data.zip
Dancer_train_2.tfrecords  data/


这里是展示已分类的模型的代码。

In [ ]:
test_io_params = {
    'is_training': False,
    'sloppy': False,
    'shuffle': True,
}
test_tfrecords = test_data_files

# 计算图？
input_graph = tf.Graph()
with input_graph.as_default():
  mesh_load_op = dataio.create_input_from_dataset(
      dataio.create_dataset_from_tfrecords, test_tfrecords, test_io_params)
  with tf.Session() as sess:
    test_mesh_data, test_labels = sess.run(mesh_load_op)

input_mesh_data = {
    'vertices': test_mesh_data['vertices'][0, ...],
    'faces': test_mesh_data['triangles'][0, ...],
    'vertex_colors': mesh_viewer.SEGMENTATION_COLORMAP[test_labels[0, ...]],
}
input_viewer = mesh_viewer.Viewer(input_mesh_data)

## Test model & visualize results

Now that we have defined the model, let's load the weights from the trained model downloaded above and use tf.Estimator.predict to predict the part labels for meshes in the test dataset.

In [ ]:
test_io_params = {
    'is_training': False,
    'sloppy': False,
    'shuffle': True,
    'repeat': False
}
test_tfrecords = test_data_files

def predict_fn():
  return dataio.create_input_from_dataset(dataio.create_dataset_from_tfrecords,
                                          test_tfrecords,
                                          test_io_params)


estimator = tf.estimator.Estimator(model_fn=model_fn,
                                   model_dir=local_model_dir,
                                   params=MODEL_PARAMS)
test_predictions = estimator.predict(input_fn=predict_fn)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/root/.keras/datasets/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Run the following cell repeatedly to cycle through the meshes in the test sequence. The left view shows the input mesh, and the right view shows the predicted part labels.

In [ ]:
prediction = next(test_predictions)
input_mesh_data = {
    'vertices': prediction['vertices'],
    'faces': prediction['triangles'],
}
predicted_mesh_data = {
    'vertices': prediction['vertices'],
    'faces': prediction['triangles'],
    'vertex_colors': mesh_viewer.SEGMENTATION_COLORMAP[prediction['predictions']],
}

input_viewer = mesh_viewer.Viewer(input_mesh_data)
prediction_viewer = mesh_viewer.Viewer(predicted_mesh_data)